# Transformers

Somewhat basic implemention of transformer model

In [32]:
#| default_exp models.transformer

In [38]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
#| export
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.optim import SGD
from torchtext.vocab import vocab

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from typing import Dict, List, Tuple, Optional, Set
from collections import Counter, OrderedDict
from dataclasses import dataclass, asdict

from plum import dispatch

from nimrod.models.lm import Vocab

## Data formatting

In [75]:
# !head ../data/en/wikitext-2/wiki.train.tokens
# read unstructured text into pd
df = pd.read_fwf('../data/en/tiny_shakespeare.txt', header=None)
print(df.head())

v = Vocab(df[0])
print(len(v), ''.join(v.vocabulary))

print(df[0][0])
txt = list(df[0][0])
print(txt, len(txt))
print(v.stoi(txt), len(v.stoi(txt)))
print(v.stoi('<pad>'))
data = df[0].tolist()

                                               0
0                                 First Citizen:
1  Before we proceed any further, hear me speak.
2                                           All:
3                                  Speak, speak.
4                                 First Citizen:
68  !$&',-.3:;<bos><eos><pad><unk>?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
First Citizen:
['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':'] 14
[52, 13, 11, 10, 6, 4, 40, 13, 6, 13, 60, 5, 12, 27] 14
0


In [78]:

# def make_dataset(names:List[str], verbose:bool=False, pad_value=0, context_length=3):
#     X = []
#     y = []
#     return torch.tensor(X),torch.tensor(y)
for i in range(2):
    print(list(data[i]))
    print(list(data[i])[])


['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':']
['B', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p', 'r', 'o', 'c', 'e', 'e', 'd', ' ', 'a', 'n', 'y', ' ', 'f', 'u', 'r', 't', 'h', 'e', 'r', ',', ' ', 'h', 'e', 'a', 'r', ' ', 'm', 'e', ' ', 's', 'p', 'e', 'a', 'k', '.']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()